# A3

## Packages
### Used Packages 
* [tslearn](https://tslearn.readthedocs.io/en/stable/quickstart.html)
* Numpy
* Pandas
* SKlearn

### Not currently used packages, but might use
* [DBA: Dynamic Time Warping Barycenter Averaging](https://github.com/fpetitjean/DBA)

## References
* [OpenFace 2.0](https://github.com/TadasBaltrusaitis/OpenFace/wiki): Facial Behavior Analysis Toolkit Tadas Baltrušaitis, Amir Zadeh, Yao Chong Lim, and Louis-Philippe Morency, IEEE International Conference on Automatic Face and Gesture Recognition, 2018
* Cassisi, C., Montalto, P., Aliotta, M., Cannata, A., & Pulvirenti, A. (2012). _Similarity measures and dimensionality reduction techniques for time series data mining_. Advances in data mining knowledge discovery and applications, 71-96.
* McKinney, W. (2012). _Python for data analysis: Data wrangling with Pandas, NumPy, and IPython_. " O'Reilly Media, Inc.".
* Shokoohi-Yekta, M., Hu, B., Jin, H., Wang, J., & Keogh, E. _Generalizing Dynamic Time Warping to the Multi-Dimensional Case Requires an Adaptive Approach (SDM 2015)_. In 2015 SIAM International Conference on Data Mining.-http://www. cs. ucr. edu/~ eamonn/Multi-Dimensional_DTW_Journal. pdf (last access: 18.12. 2015).
* [TsLearn](https://tslearn.readthedocs.io/en/stable/gettingstarted.html) documentation


In [16]:
%ls

A3.ipynb               backup/                utilities/
CreateDataset.ipynb    media/                 venv/
GIFGIF_Dataset/        requirements.txt
GIFGIF_Download.ipynb  run_openface.py


## Imports

In [17]:
import pandas
import numpy
import glob
from typing import *
import umap

from sklearn import preprocessing

import tslearn.utils as tsutils
from tslearn.metrics import dtw
from tslearn.clustering import TimeSeriesKMeans, silhouette_score

import matplotlib.pyplot as plt

## Regenerate all data
set `REGENERATE` to true if you want to regenerate all data. If set to false, data saved to disk will be used. Regenerating all the data may be time-consuming, so be warned.

In [ ]:
# Can take up to 1 hour
REGENERATE_DATASET = False

# Can take up to 2 hours
REGENERATE_BIC = False

## Dataset creation

As I am going through my semester working through 4 higher-division classes and 20 hours of part-time work per week, my emotions have shifted from peace to a combination of sadness, anger and powerlessness. In order to keep my attitude positive, I chose the better of the three: sadness. 

I was looking to run OpenFace on the gifs, but that is apparently not possible. After spending enough hours trying to install OpenFace on my machine through 3 different methods given in the Wiki (native MacOS, `bamos/openface` docker image, `algebr/openface` docker image and creating my own docker version), I decided to go with the existing docker image from `algebr/openface`, using the method [described here](https://github.com/TadasBaltrusaitis/OpenFace/wiki). I had issues with all of the other methods.

I converted all gifs to images, copied them to the docker container and and ran OpenFace on the sequences of images.
```bash=
docker run -it -d --rm algebr/openface:latest
docker cp images 3a73fbce562e:/home/openface-build
docker exec -it 3a73fbce562e sh
docker cp run_openface.py 3a73fbce562e:/home/openface-build
```
I then ran my `run_openface.py` script to create the CSVs out of the image folders, and copied the results to my local machine:
```bash
docker cp  40b27:/home/openface-build/output ./output
```

My first look at the processed data seem to imply that OpenFace failed to parse all of the cartoons. It did show partial information, but instances of cartoons may need to be dropped from the dataset. Only images of real humans were fully labelled.

_NOTE: not sure how to submite my assignment with my dataset as it is 2GB_

## Approach

I am not certain how to approach a multivariate DTW problem. It seems like there are two main approaches [(2015, Shokoohi-Yekta et al)](https://link.springer.com/article/10.1007/s10618-016-0455-0):
1. Independent DTW: find the distance between every dimension for two time series. For multi-dimensional time series (MDT) Q and C, $$DTW_1(Q, C) = \sum_{m=1}^{M} DTW(Q_m, C_m)$$ Each dimension is independent.
2. Dependent DTW: warp all dimensions in a single warping matrix. The dimensions are now dependent. It's similar to single-dimension DTW, except the distance is measured for M data-points. The following distance function is used: $$d(q_i, c_j) = \sum_{m=1}^{M} DTW(q_{i, m}, c_{j, m})^2$$

### Areas of interest in the data
[See here for in-depth explanations about output format of OpenFace](https://github.com/TadasBaltrusaitis/OpenFace/wiki/Output-Format)

[OpenFace action units information](https://github.com/TadasBaltrusaitis/OpenFace/wiki/Action-Units)

#### All columns in the CSVs



| Column                         | Description                                                  | Notes                                                        | Selected |
| ------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | -------- |
| `gaze_0_x, gaze_0_y, gaze_0_z` | eye 0, leftmost eye in image                                 |                                                              | N        |
| `gaze_1_x, gaze_1_y, gaze_1_z` | Eye 1, rightmost eye in image                                |                                                              | N        |
| `gaze_angle_x, gaze_angle_y`   | Eye gaze direction in radians in world coordinates averaged for both eyes and converted into more easy to use format than gaze vectors | Might want to use that over `gaze`, as `gaze` is containted in it. | Y        |
| `eye_lmk_x*, eye_lmk_X*`       | location of 3D eye in pixels (x) and mm (X)                  | Redundant if using head rotation and translation             | N        |
| `pose_Rx, pose_Ry, pose_Rz`    | pitch, yaw and roll for head in radians in relation to camera | Head rotation may be useful                                  | Y        |
| `pose_Tx, pose_Ty, pose_Tz`    | location of the head with respect to camera in millimeters   | Head translation may be useful                               | Y        |
| `AU_r`                         | Action units intesity of 17 AUs (from 0 to 5)                | Definitely useful                                            | Y        |
| `AU_c`                         | Action unit presence of 18 AUs                               | Definitely useful                                            | Y        |

#### Fourier Transform
Should I apply a Fourier transform on the time series, and run DTW on the output, or should I simply run DTW on the current time series?

#### Number of features
The model may overfit if too many features are provided. Try at least 3 different amount of features.

## Infrastructure

### Preprocess time series 
* Normalize to have zero mean and unit variance
* [Resample?](https://tslearn.readthedocs.io/en/stable/gen_modules/preprocessing/tslearn.preprocessing.TimeSeriesResampler.html#tslearn.preprocessing.TimeSeriesResampler)

Apply UMAP to related dimensions to decrease number of dimensions.

In [19]:
# Make UMAP reproducible by using the same reducer with a set random state
reducer = umap.UMAP(random_state=42, n_components=2)

def apply_umap(pd: List[List]):
    """Return a table with less columns by reducing dimensions"""
    # Apply UMAP
    # print(pd.head())
    reducer.fit(pd)
    embedding = reducer.transform(pd)
    print(f"New embedding column number: {len(embedding[0, :])}")
    return pandas.DataFrame.from_records(embedding)

def preprocess(df, AU_only) -> List[Any]:
    """
    * Normalize to have zero mean and unit variance for every dimension.
    * Select wanted dimensions.
    """
    if AU_only:
        df = df[[
                    "AU01_r",
                    "AU02_r",
                    "AU04_r",
                    "AU05_r",
                    "AU06_r",
                    "AU07_r",
                    "AU09_r",
                    "AU10_r",
                    "AU12_r",
                    "AU14_r",
                    "AU15_r",
                    "AU17_r",
                    "AU20_r",
                    "AU23_r",
                    "AU25_r",
                    "AU26_r",
                    "AU45_r",
                    "AU28_c" # This one not contained in AU*_r
        ]].copy()
    else:
        df = df[[
                    "gaze_angle_x",
                    "gaze_angle_y",
                    "pose_Rx",
                    "pose_Ry",
                    "pose_Rz",
                    "pose_Tx",
                    "pose_Ty",
                    "pose_Tz",
                    "AU01_r",
                    "AU02_r",
                    "AU04_r",
                    "AU05_r",
                    "AU06_r",
                    "AU07_r",
                    "AU09_r",
                    "AU10_r",
                    "AU12_r",
                    "AU14_r",
                    "AU15_r",
                    "AU17_r",
                    "AU20_r",
                    "AU23_r",
                    "AU25_r",
                    "AU26_r",
                    "AU45_r",
                    "AU28_c" # This one not contained in AU*_r
        ]].copy()
    
    df = pandas.DataFrame(preprocessing.scale(df), columns=df.columns)
    
    # Let's apply UMAP to the whole time series at once
    df_new = apply_umap(df)

    return df_new

In [20]:
def centroids_prototype(centroids, X):
    """Return one or more prototype for each centroid"""
    prototypes = {i: {"dist": float("inf"), "index": 0} for i in range(len(centroids))}

    for i, _ in enumerate(centroids):
        for j, _ in enumerate(X):
            dist = dtw(centroids[i], X[j])
            if dist < prototypes[i]["dist"]:
                prototypes[i] = {"dist": dist, "index": j}
    return prototypes

def centroids_prototypes(centroids, X, num_protos=4):
    """Return more than one prototype for each centroid"""
    if num_protos == 1:
        print("number of centroids must be greater than 1")
        return
    prototypes: List[Tuple(float, int)] = []

    for i, _ in enumerate(centroids):
        tmp = []
        for j, _ in enumerate(X):
            dist = dtw(centroids[i], X[j])
            tmp.append((dist, j))
        tmp.sort(key=lambda tup: tup[0])
        prototypes.append(tmp[:num_protos])
            
    return prototypes

### Create time series datasets

In [21]:
CSV_EXT = ".csv"
TIME_SERIES_DATASETS_PATH = "GIFGIF_Dataset/time_series_datasets"

def extract_time_series_dataset(path: str, AU_only=False) -> List[Any]:
    """Create a time series dataset from all CSVs located at in 
    a directory
    :param path: path of the directory
    """
    if path[-1] == "/":
        path = path[:-1]
    dirs = glob.glob(f"{path}/*")

    # Save every gif's openface dataset in a pandas dataframe.
    time_series_list = []
    
    backup_file_name = path.split("/")[-1]
    backup_file_path = f"{TIME_SERIES_DATASETS_PATH}/{backup_file_name}.txt"

    print(f"Creating dataset for {path}")

    # Keep track of number of created time series
    created_count = 0
    # time_series_labels show the path to the OpenFace dir
    time_series_labels = []

    for dir in dirs:
        filename = dir.split("/")[-1]
        csv_filepath = f"{dir}/{filename}{CSV_EXT}"
        with open(csv_filepath) as f:
            df = pandas.read_csv(f, dtype=float)

            # Column names sometimes have an extra space on the left or right
            df.rename(columns=lambda x: x.strip(), inplace=True)

            if df.loc[:, "confidence"].mean() < 0.8:
                continue

            # Remove all NA rows
            df.dropna()

            # Select only wanted columns and scale
            df_new = preprocess(df, AU_only)

            formatted_time_series = tsutils.to_time_series(df_new)
            time_series_list.append(formatted_time_series)
            time_series_labels.append(dir)
            created_count += 1

    time_series_dataset = tsutils.to_time_series_dataset(time_series_list)

    print(f"Saving dataset for {path} to file {backup_file_path}")
    tsutils.save_time_series_txt(backup_file_path, time_series_dataset)
    print(f"Created {created_count} time series for time series dataset")
    return time_series_dataset, time_series_labels

## Load Dataset
Load the generated CSVs into a pandas dataframe.

In [29]:
%%time
%%capture
BACKUP = "backup"

OPENFACE_DATA = "GIFGIF_Dataset/openface"

X_file = f"{BACKUP}/X"
X_labels_file = f"{BACKUP}/X_labels"

X = None
X_labels = None
    
if REGENERATE_DATASET:
    X, X_labels = extract_time_series_dataset(f"{OPENFACE_DATA}/sadness/", AU_only=False)
    
    numpy.save(X_file, X)
    numpy.save(X_labels_file, X_labels)
else:
    # Read from file
    X = numpy.load(X_file + ".npy")
    X_labels = numpy.load(X_labels_file + ".npy")

CPU times: user 3.84 ms, sys: 2.05 ms, total: 5.89 ms
Wall time: 7.55 ms


In [ ]:
%%time
%%capture

X_AU_only_file = f"{BACKUP}/X_AU_only"
X_AU_only_labels_file = f"{BACKUP}/X_AU_only_labels"

X_AU_only = None
X_AU_only_labels = None

if REGENERATE_DATASET:
    X_AU_only, X_AU_only_labels = extract_time_series_dataset(f"{OPENFACE_DATA}/sadness/", AU_only=True)
    numpy.save(X_AU_only_file, X_AU_only)
    numpy.save(X_AU_only_labels_file, X_AU_only_labels)
else:
    # Read from file
    X_AU_only = numpy.load(X_AU_only_file + ".npy")
    X_AU_only_labels = numpy.load(X_AU_only_labels_file + ".npy")

# Silhouette Coefficients
The higher the silhouette coefficient, the better the clustering. The silhouette coefficient [is defined as](https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient):
* a: The mean distance between a sample and all other points in the same class.
* b: The mean distance between a sample and all other points in the next nearest cluster.
The silhouette coefficient for a sample $s$ is given by: $$s = \frac{b-a}{max(a, b)}$$
The silhouette coefficient for a set is the mean of the silhouette coefficient of every member in that set.

More info on silhouette coefficients from [sklearn](https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient).

In [ ]:
%%capture
%%time

def calculate_BIC(X, labels):
    num_clusters = numpy.arange(2, 14)
    sil: List[float] = []
        
    for num_cluster in num_clusters:
        km = TimeSeriesKMeans(n_clusters=num_cluster, metric="dtw").fit(X)
        labels = km.labels_
        score = silhouette_score(X, labels, metric="dtw")
        sil.append(score)
    
    return sil, X, labels

In [ ]:
MEDIA = "media"

def plt_BIC(sil, X, labels):
    plt.plot(num_clusters, sil)
    plt.title('Silhouette Scores')
    plt.ylabel('silhouette score')
    plt.xlabel('number of cluster')
    plt.grid(True)
    # Save plot
    plt.savefig(f"{MEDIA}/silhoutte_score.png")
    plt.show()


In [ ]:
if REGENERATE_BIC:
    plt_BIC(calculate_BIC(X, X_labels))

In [ ]:
if REGENERATE_BIC:
    plt_BIC(calculate_BIC(X_AU_only, X_AU_only_labels))

<table>
  <tr>
    <th><img src="media/silhoutte_score.png" alt="Silhouette Scores for 2 to 10 clusters"></th>
  </tr>
</table>

In [ ]:
%%time
%%capture
km_2_clusters = TimeSeriesKMeans(n_clusters=2, metric="dtw").fit(X)

In [ ]:
%%time
%%capture
km_4_clusters = TimeSeriesKMeans(n_clusters=4, metric="dtw").fit(X)

## Prototype for each centroid
I am not certain my method is correct here. I kept the 2 and 4 KMeans models to look at the prototype of each centroid. What I call prototype is the point closest to each centroid, it is the best representant of that centroid.

### 2 clusters
TODO: DESCRIBE THE EMOTIONS

In [ ]:
centroids_2 = km_2_clusters.cluster_centers_
prototypes_2 = centroids_prototype(centroids_2, X)
multiple_prototypes_2 = centroids_prototypes(centroids_2, X)

In [ ]:
print(multiple_prototypes_2)

OpenFace was set to detect single faces only. Gifs that include more than one face only contain landmarks and action units for one face. An OpenFace aligned image is provided to determine which person is being classified.

In [ ]:
# import matplotlib.image as mpimg
from IPython.display import Image, Video


GIFS_SADNESS = "GIFGIF_Dataset/gifs"

def prototype_images(prototypes):
    # Getting dict of single proto
    if type(prototypes[0]) == dict:
        for key, _ in enumerate(prototypes):
                print(f"distance: {prototypes[key]['dist']}")
                # Get OpenDace data path from prototype index
                openface_dir = X_labels[(prototypes[key]["index"])]
                gif_name = openface_dir.split("/")[-1]
                print(gif_name)

                gif_path = f"{GIFS_SADNESS}/{gif_name}.gif"

                with open(gif_path,'rb') as file:
                    display(Image(file.read()))
                print(f"{openface_dir}/{gif_name}.avi")
                with open(f"{openface_dir}/{gif_name}_aligned/frame_det_00_000001.bmp",'rb') as file:
                    display(Image(file.read()))
                    
    # Getting list of multiple protos
    else:
        for key, cluster in enumerate(prototypes):
            print("\n####################################################################")
            print(f"CLUSTER {key}")
            print("####################################################################")
            for example in cluster:
                
                print(f"distance: {example[0]}")
                # Get OpenDace data path from prototype index
                openface_dir = X_labels[example[1]]
                gif_name = openface_dir.split("/")[-1]
                gif_path = f"{GIFS_SADNESS}/{gif_name}.gif"

                with open(gif_path,'rb') as file:
                    display(Image(file.read()))
                print(f"{openface_dir}/{gif_name}.avi")
                with open(f"{openface_dir}/{gif_name}_aligned/frame_det_00_000001.bmp",'rb') as file:
                    display(Image(file.read()))
                
                
#prototype_images(prototypes_2)
prototype_images(multiple_prototypes_2)

### 4 clusters

In [ ]:
centroids_4 = km_4_clusters.cluster_centers_
prototypes_4 = centroids_prototype(centroids_4, X)
multiple_prototypes_4 = centroids_prototypes(centroids_4, X)

# prototype_images(prototypes_4)
prototype_images(multiple_prototypes_4)